### summary_request1,2
### web crawling
- 웹 서비스의 구조 : 서버 - 클라이언트
- crwaling 방법 3가지 
    - requests : json : 동적페이지
        - 네이버 주식 사이트에서 주가 데이터 크롤링
            - 시각화(정규화), 상관계수
    - requests : html : 정적페이지
    - selenium : webdriver
- 크롤링 절차
    - 웹서비스 분석 : url
    - 요청, 응답 : str
    - str 데이터 파싱(dict, bs obj(css-selector) 후 데이터 프레임 생성

### 직방
- 동이름을 입력하면 아파트 매물을 데이터 프레임으로 만들기
- robots.txt

1. 웹서비스 분석 : url 찾기, 크롤링 전략 세우기
- 동이름 입력 -> lat, lng
- lat, lng -> geohash 패키지 설치
- geohach -> item ids
- item ids -> item dats (데이터 프레임)

In [106]:
# 동이름 -> 위도경도
addr = '수진동'
url = f"https://apis.zigbang.com/search?q={addr}"
response = requests.get(url)
data = response.json()['items'][0]
lat, lng = data['lat'], data['lng']
lat, lng

(37.438575744628906, 127.13903045654297)

In [107]:
# 위도, 경도 -> geohash 위도경도를 문자형태로 바꿈
import geohash2

In [108]:
# precision : 위도경도값을 중심으로, 작을수록 큰범위, 클수록 작은 범위 
geohash = geohash2.encode(lat, lng, precision=5)
lat, lng, geohash

(37.438575744628906, 127.13903045654297, 'wydkv')

In [98]:
import itertools

In [99]:
# geohash -> ids
# 영상 대신 수정
url = f"https://apis.zigbang.com/v2/aparts/items?domain=zigbang&geohash={geohash}"
response = requests.get(url)
items = response.json()['items']
ids = [item['itemIds'] for item in items]
ids = list(itertools.chain(*ids))
len(ids), ids[:3]

(457, ['18332563', '18332967', '17530954'])

In [100]:
# 영상 내용
# # geohash -> ids
# url = f"http://apis.zigbang.com/property/apartments/items?domain=zigbang&geohash={geohash}&q=type=sales%7Cprice=0~-1%7CfloorArea=0~-1"
# response = requests.get(url)
# items = response.json()['items']
# ids = [item['itemId'] for item in items]
# len(ids), ids[:3]

In [101]:
# ids = items
url = "https://apis.zigbang.com/v2/items/list"
params = {
    'domain' : 'zigbang',
    'withCoalition' : 'false',
    'item_ids' : ids
}
response = requests.post(url,params)
items = response.json()['items']

In [102]:
# pandas 데이터프레임으로 변경
item_df = pd.DataFrame(items)
# pandas 옵션 설정 컬럼이 다 안나와서
pd.options.display.max_columns = len(item_df.columns)
item_df

,section_type,item_id,images_thumbnail,sales_type,sales_title,deposit,rent,size_m2,공급면적,전용면적,room_type_title,floor,floor_string,building_floor,title,is_first_movein,room_type,address,random_location,is_zzim,status,service_type,tags,address1,address2,address3,manage_cost,reg_date,is_new
0,premium_normal,18332563,https://ic.zigbang.com/ic/floorplan/302/665/06...,매매,매매,68000,0,None,"{'m2': 106, 'p': '32'}","{'m2': 84, 'p': '25'}","{'m2': '106A', 'p': '32A'}",저,저,17,내부깨끗.초중인접 학군아주우수함,None,None,동아1차 101동,None,False,True,아파트,[],None,None,None,None,2020-02-18T14:31:55+09:00,True
1,premium_normal,18332967,https://ic.zigbang.com/ic/floorplan/302/666/57...,매매,매매,80000,0,None,"{'m2': 144, 'p': '43'}","{'m2': 114, 'p': '34'}","{'m2': '144', 'p': '44'}",중,중,17,한강조망 .초중인접 학군아주우수함 한강조망가능,None,None,동아1차 103동,None,False,True,아파트,[],None,None,None,None,2020-02-18T14:31:55+09:00,True
2,premium_normal,17530954,https://ic.zigbang.com/ic/floorplan/302/665/06...,전세,전세,39000,0,None,"{'m2': 106, 'p': '32'}","{'m2': 84, 'p': '25'}","{'m2': '106A', 'p': '32A'}",중,중,17,"올수리.초,중학교앞",None,None,동아1차 101동,None,False,True,아파트,[],None,None,None,None,2020-02-13T11:53:16+09:00,False
3,premium_normal,17897931,https://ic.zigbang.com/ic/floorplan/302/666/57...,전세,전세,50000,0,None,"{'m2': 144, 'p': '43'}","{'m2': 114, 'p': '34'}","{'m2': '144', 'p': '44'}",중,중,17,"올수리.한강조망.초,중학교앞",None,None,동아1차 102동,None,False,True,아파트,[],None,None,None,None,2020-02-13T11:53:16+09:00,False
4,premium_normal,18474626,https://ic.zigbang.com/ic/floorplan/302/666/57...,매매,매매,85000,0,None,"{'m2': 144, 'p': '43'}","{'m2': 114, 'p': '34'}","{'m2': '144', 'p': '44'}",중,중,17,"하시입주.올수리.한강조망.초,중학교앞",None,None,동아1차 102동,None,False,True,아파트,[],None,None,None,None,2020-02-13T11:53:16+09:00,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
452,premium_normal,20522771,https://ic.zigbang.com/ic/floorplan/38615/1557...,전세,전세,80000,0,None,"{'m2': 106, 'p': '32'}","{'m2': 84, 'p': '25'}","{'m2': '106B', 'p': '32B'}",저,저,29,"급전세,중층남향,융자무,시스템에어컨",None,None,당산센트럴아이파크 101동,None,False,True,아파트,[],None,None,None,None,2020-02-19T17:07:23+09:00,True
453,premium_normal,20501315,https://ic.zigbang.com/ic/floorplan/38615/1557...,월세,월세,80000,60,None,"{'m2': 140, 'p': '42'}","{'m2': 114, 'p': '34'}","{'m2': '140', 'p': '42'}",저,저,28,"귀한월세,남향,",None,None,당산센트럴아이파크 102동,None,False,True,아파트,[],None,None,None,None,2020-02-18T16:38:20+09:00,True
454,premium_normal,20411815,https://ic.zigbang.com/ic/floorplan/38615/1557...,전세,전세,55000,0,None,"{'m2': 65, 'p': '19'}","{'m2': 46, 'p': '13'}","{'m2': '65', 'p': '20'}",저,저,24,"급전세,신축아파트,,융자무",None,None,당산센트럴아이파크 107동,None,False,True,아파트,[],None,None,None,None,2020-02-13T18:09:33+09:00,False
455,premium_normal,20411769,https://ic.zigbang.com/ic/floorplan/38615/1557...,전세,전세,72000,0,None,"{'m2': 79, 'p': '23'}","{'m2': 59, 'p': '17'}","{'m2': '79A', 'p': '24A'}",중,중,29,"급전세,신축아파트,융자무",None,None,당산센트럴아이파크 106동,None,False,True,아파트,[],None,None,None,None,2020-02-13T18:09:31+09:00,False


In [83]:
# 필요한 컬럼만 정리
columns = ['item_id','floor','building_floor','address','sales_type','rent', 'deposit','전용면적']
item_df = item_df[columns].copy()
item_df.tail()

,item_id,floor,building_floor,address,sales_type,rent,deposit,전용면적
452,20522771,저,29,당산센트럴아이파크 101동,전세,0,80000,"{'m2': 84, 'p': '25'}"
453,20501315,저,28,당산센트럴아이파크 102동,월세,60,80000,"{'m2': 114, 'p': '34'}"
454,20411815,저,24,당산센트럴아이파크 107동,전세,0,55000,"{'m2': 46, 'p': '13'}"
455,20411769,중,29,당산센트럴아이파크 106동,전세,0,72000,"{'m2': 59, 'p': '17'}"
456,20030556,저,28,당산센트럴아이파크 104동,전세,0,70000,"{'m2': 71, 'p': '21'}"


In [84]:
# 평수 적기
item_df['m2'] = item_df['전용면적'].apply(lambda data: data['m2'])
item_df['p'] = item_df['전용면적'].apply(lambda data: data['p'])
item_df.drop(columns=['전용면적'],inplace=True)
item_df.tail()

,item_id,floor,building_floor,address,sales_type,rent,deposit,m2,p
452,20522771,저,29,당산센트럴아이파크 101동,전세,0,80000,84,25
453,20501315,저,28,당산센트럴아이파크 102동,월세,60,80000,114,34
454,20411815,저,24,당산센트럴아이파크 107동,전세,0,55000,46,13
455,20411769,중,29,당산센트럴아이파크 106동,전세,0,72000,59,17
456,20030556,저,28,당산센트럴아이파크 104동,전세,0,70000,71,21


#### 함수로 만들기
- 동이름 -> 아파트 매물 데이터 프레임

In [85]:
def crawling_apt(addr) :
    # 동이름 -> 위도 경도
    
    url = f"https://apis.zigbang.com/search?q={addr}"
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    # 위도 경도 -> geohash
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # geohash -> ids

    url = f"https://apis.zigbang.com/v2/aparts/items?domain=zigbang&geohash={geohash}"
    response = requests.get(url)
    items = response.json()['items']
    ids = [item['itemIds'] for item in items]
    ids = list(itertools.chain(*ids))
    
    # ids -> items
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        'domain' : 'zigbang',
        'withCoalition' : 'false',
        'item_ids' : ids
    }
    response = requests.post(url,params)
    items = response.json()['items']
    item_df = pd.DataFrame(items)
    columns = ['item_id','floor','building_floor','address','sales_type','rent', 'deposit','전용면적']
    item_df = item_df[columns].copy()
    item_df['m2'] = item_df['전용면적'].apply(lambda data: data['m2'])
    item_df['p'] = item_df['전용면적'].apply(lambda data: data['p'])
    item_df.drop(columns=['전용면적'],inplace=True)
    
    return item_df
    

In [61]:
result = crawling_apt('성수동')
result.tail()

,item_id,floor,building_floor,address,sales_type,rent,deposit,m2,p
403,19940734,저,21,e편한세상금호파크힐스 107동,월세,100,50000,84,25
404,19940917,저,21,e편한세상금호파크힐스 105동,전세,0,75000,84,25
405,20302198,중,8,e편한세상금호파크힐스 116동,전세,0,60000,59,17
406,20300675,저,19,e편한세상금호파크힐스 101동,전세,0,74000,84,25
407,20132279,저,21,e편한세상금호파크힐스 106동,월세,190,10000,59,17


In [62]:
# 도곡동에서 30층 이상되는 아파트에 고층 매물 찾기

In [63]:
result = crawling_apt('도곡동')
result.tail()

,item_id,floor,building_floor,address,sales_type,rent,deposit,m2,p
149,19963421,중,25,신반포자이 107동,전세,0,120000,59,17
150,20429678,중,15,마일스디오빌 1동,전세,0,33500,36,10
151,20096477,중,15,마일스디오빌 1동,월세,90,5000,36,10
152,20096352,저,15,마일스디오빌 1동,월세,120,500,36,10
153,20097031,None,15,마일스디오빌 1동,매매,0,38000,36,10


In [49]:
#result.dtypes
result['building_floor'] = result['building_floor'].astype('int64')

In [58]:
result = result[
    (result['building_floor'] >= 30)
    & (result['floor'] == '중')
    & (result['sales_type'] == '매매')
]
result

,item_id,floor,building_floor,address,sales_type,rent,deposit,m2,p
112,20479984,중,35,반포리체 106동,매매,0,195000,59,17
135,20480154,중,34,서초푸르지오써밋 105동,매매,0,245000,84,25
137,20391106,중,30,서초푸르지오써밋 202동,매매,0,295000,120,36


In [60]:
%%writefile zigbanghm.py
import pandas as pd
import requests
import geohash2
import itertools

def crawling_apt(addr) :
    # 동이름 -> 위도 경도
    
    url = f"https://apis.zigbang.com/search?q={addr}"
    response = requests.get(url)
    data = response.json()['items'][0]
    lat, lng = data['lat'], data['lng']
    
    # 위도 경도 -> geohash
    geohash = geohash2.encode(lat, lng, precision=5)
    
    # geohash -> ids

    url = f"https://apis.zigbang.com/v2/aparts/items?domain=zigbang&geohash={geohash}"
    response = requests.get(url)
    items = response.json()['items']
    ids = [item['itemIds'] for item in items]
    ids = list(itertools.chain(*ids))
    
    # ids -> items
    url = "https://apis.zigbang.com/v2/items/list"
    params = {
        'domain' : 'zigbang',
        'withCoalition' : 'false',
        'item_ids' : ids
    }
    response = requests.post(url,params)
    items = response.json()['items']
    item_df = pd.DataFrame(items)
    columns = ['item_id','floor','building_floor','address','sales_type','rent', 'deposit','전용면적']
    item_df = item_df[columns].copy()
    item_df['m2'] = item_df['전용면적'].apply(lambda data: data['m2'])
    item_df['p'] = item_df['전용면적'].apply(lambda data: data['p'])
    item_df.drop(columns=['전용면적'],inplace=True)
    
    return item_df

Overwriting zigbanghm.py
